# Illustrates how to sniff USB traffic and interpret it with ViewSB:

### Connect:

In [ ]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=True)
phy.set_power_source("host")

### Check bitfile build time:
During development it's handy to confirm that the newest bitfile is being used!

In [ ]:
print(phy.get_fpga_buildtime())

### Power-off target to allow PhyWhisperer to autodetect its speed:
Ensure the target device is connected to the PhyWhisperer.
We'll turn off the target's power so that the PhyWhisperer can be programmed before turning the target back on. We do this because we'll be capturing what the target does when it's first turned on.

In [ ]:
import time
phy.set_power_source("off")
time.sleep(0.5)
phy.reset_fpga()
phy.set_usb_mode('auto')

### Tell PhyWhisperer how many events to capture:
Maximum is 8188.

In [ ]:
phy.set_capture_size(8188)

### Arm the PhyWhisperer:
You should see the blue ARM LED turn on to reflect the armed status.
In this example we don't want to generate an external trigger, we only want to capture the USB traffic.

In [ ]:
phy.arm()
phy.set_trigger(enable=False)

### Program the pattern match:

In [ ]:
phy.set_pattern(pattern=[0x2d, 0x00], mask=[0xff, 0xff])

### Power up the target:
Now that PhyWhisperer is programmed, power up the target. PW should auto-detect the correct speed, and the capture should be triggered.

In [ ]:
phy.set_power_source("host")
#Let device enumerate
time.sleep(1.0)

### Ensure correct USB speed was detected:
If the assertion fails, try setting the USB speed manually with set_usb_mode().

In [ ]:
assert (phy.get_usb_mode() == 'FS')
#assert (phy.get_usb_mode() == 'LS')
#assert (phy.get_usb_mode() == 'HS')

### Read what was captured:

In [ ]:
raw = phy.read_only_from_fifo(entries=8188)

### Check the capture memory status:
No overflow or underflow events should have occured.

In [ ]:
phy.check_fifo_errors()

### Interpret the captured data:
The pattern match byte which triggered the captured isn't recorded; let's add it back it so that all of the captured USB data can be properly interpreted:

In [ ]:
phy.addpattern = True

Then we split the raw captured data and timestamps into packets:

In [ ]:
packets = phy.split_packets(raw)

Now we can use ViewSB to interpret the packets:

In [ ]:
printPackets = pw.USBSimplePrintSink(highspeed=phy.get_usb_mode() == 'HS')

for packet in packets:
    printPackets.handle_usb_packet(ts=packet['timestamp'], buf=bytearray(packet['contents']), flags=0)

### Illustrate USB status monitor feature:
PhyWhisperer can be programmed to log a match on any of the 5 USB status lines:
- rx active (bit 0)
- rx error (bit 1)
- session valid (bit 2)
- session end (bit 3)
- vbus valid (bit 4)

In glitching attacks it may be useful to monitor rxerror.
In this example we're not attacking the target, so let's just illustrate a match event on the 'session end' status line:

In [ ]:
# look only at bit 3 (session end), don't care what the other bits are:
phy.set_stat_pattern(pattern=0x8, mask=0x8)
# power off target to cause a "session end" event:
phy.set_power_source('off')
assert phy.stat_pattern_matched() == 1, "oops, didn't see a session end event!"
print("Matched USB status lines: %s" % hex(phy.stat_pattern_match_value))

In [ ]:
phy.close()